### Intsalls / Imports

In [ ]:
!pip install asyncpraw --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 5.4 MB/s eta 0:00:00


In [ ]:

import asyncpraw
import pandas as pd
import asyncio
import os

from google.colab import drive
drive.mount('/content/drive/')

### Scrape Web Data

In [ ]:
# Define an asynchronous function to get more rows of posts by paginating
async def get_more_posts(subreddit_list='MachineLearning', limit=400000):

    # Create a read-only Reddit instance using asyncpraw
    reddit = asyncpraw.Reddit(client_id='Y71Q4eduaps7rkUBJFFN8w',
                              client_secret='bZnZvPnUnkdyke3ngUnFIpWDFlil1A',
                              redirect_uri='http://localhost:8080',
                              user_agent='266_MentalHealth')

    subreddit = await reddit.subreddit(subreddit_list)

    posts_df = []
    count = 0
    after = None

    while count < limit:
        posts = subreddit.new(limit=1000, params={'after': after})

        batch_count = 0
        async for post in posts:
            if post.is_self:
                posts_df.append({'subreddit': post.subreddit.display_name,
                                 'selftext': post.selftext,
                                 'date': post.created_utc,
                                 'author': post.author.name if post.author else None
                                 })
                batch_count += 1
                count += 1

            if count >= limit:
                break

        if batch_count > 0:
            after = post.id
        else:
            break

    return pd.DataFrame(posts_df)

# Reddit Mental Health Data parameteres
spec_mental_health = 'EDAnonymous+addiction+alcoholism+adhd+anxiety+autism+bipolarreddit+bpd+depression+healthanxiety+lonely+ptsd+schizophrenia+socialanxiety+suicidewatch'
broad_mental_health = 'mentalhealth+COVID19_support'
non_mental_health = 'conspiracy+divorce+fitness+guns+jokes+legaladvice+meditation+parenting+personalfinance+relationships+teaching'

subreddits = spec_mental_health + '+' + broad_mental_health + '+' + non_mental_health

web_posts_df = await get_more_posts(subreddit_list=subreddits, limit=400000)
web_posts_df = web_posts_df.rename(columns={'selftext':'post',})

In [ ]:
web_posts_df.head()

,subreddit,post,date,author
0,legaladvice,I am not elderly so I am not sure if an elder ...,1.730240e+09,FirebirdWriter
1,legaladvice,I requested this by filing a motion and would ...,1.730240e+09,AttemptMysterious889
2,autism,Hi. Please don’t get mad at the title. I just ...,1.730240e+09,Soft-Funny-689
3,SuicideWatch,Cycling between hating myself for hurting her ...,1.730240e+09,big_penguin_problems
4,legaladvice,This is an odd one but looking for serious hel...,1.730240e+09,Maddieann24


In [ ]:
web_posts_df.shape

(400000, 4)

In [ ]:
web_posts_df['subreddit'].value_counts()

,count
subreddit,
legaladvice,71572
ADHD,35880
SuicideWatch,33000
Anxiety,29246
depression,23989
autism,23094
personalfinance,22565
mentalhealth,21044
lonely,20782


### Load Pre/Post Datasets

In [3]:
PATH = '/content/drive/MyDrive/DS 266/project_data/'
files = os.listdir(PATH)

In [4]:
project_data = []
columns = ['subreddit','author','date','post']
files = files[1:]

for i in range(len(files)):
  if files[i] != 'final data':
    NEW_PATH = PATH + files[i]
    temp_df = pd.read_csv(NEW_PATH)

    project_data.append(temp_df[columns])

project_df = pd.concat(project_data)
project_df.head()

,subreddit,author,date,post
0,addiction,iephillips,2019/04/16,Heroin Should I tell my ex wife that I tried h...
1,addiction,michaelmichael861,2019/04/16,"Strange side effects, please give any advice. ..."
2,addiction,waismannmethod,2019/04/16,New York Heroin Use Figures Higher Than US Ave...
3,addiction,StunnyG,2019/04/16,New here but not to recovery. Looking to share...
4,addiction,8923578295789advice,2019/04/16,im clean my aunt (who i live with) found a pac...


In [5]:
project_df.to_csv(PATH + 'final data/project_data01.csv', index=False)

In [ ]:
project_df['subreddit'].value_counts()

,count
subreddit,
legaladvice,80757
personalfinance,60605
depression,59242
jokes,46249
anxiety,35872
relationships,35804
mentalhealth,32439
fitness,32193
adhd,30298


### Combine and Save Data to 'csv'

In [ ]:
project_df = pd.concat([project_df, web_posts_df])
project_df.to_csv(PATH + 'final data/project_data.csv', index=False)
project_df.head()

,subreddit,author,date,post
0,addiction,MushroomEagle,2020/01/01,Hadn’t even made it a day Just relapsed for th...
1,addiction,karynlackey,2020/01/01,I read my way out of addiction and into a mast...
2,addiction,throwaway2463684,2020/01/01,I think my friend relapsed I haven't known her...
3,addiction,Christiannan,2020/01/01,"It’s now been 8 years since I quit opiates, co..."
4,addiction,k_thrace_,2020/01/01,First sober NYE in (at least) a decade! Last n...


In [ ]:
project_df.shape

(1016648, 4)

In [ ]:
project_df['subreddit'].value_counts()

,count
subreddit,
legaladvice,152329
depression,83231
personalfinance,83170
mentalhealth,53483
relationships,52594
jokes,46249
lonely,40282
ADHD,35880
anxiety,35872
